Database Credentials


In [1]:
import os

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
import os
from dotenv import load_dotenv
from langchain_community.utilities.sql_database import SQLDatabase

# Load environment variables
load_dotenv()

# Get database credentials from environment variables
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")

# Create database connection
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}", schema=db_name, view_support=True)
print(db.dialect)
print(db.get_usable_table_names())
print(db.table_info)

mysql
['ad_creatives', 'ad_insights', 'ads', 'campaign_actions', 'campaign_conversions', 'campaign_insights', 'campaigns']

CREATE TABLE adsnerd.ad_creatives (
	id BIGINT NOT NULL, 
	ad_id BIGINT NOT NULL, 
	name VARCHAR(255), 
	title VARCHAR(255), 
	call_to_action_type VARCHAR(50), 
	PRIMARY KEY (id, ad_id), 
	CONSTRAINT ad_creatives_ibfk_1 FOREIGN KEY(ad_id) REFERENCES adsnerd.ads (id)
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from ad_creatives table:
id	ad_id	name	title	call_to_action_type
23859344584770310	23859323743330310	Become the industry's go-to marketer. Subscribe now! 2023-09-14-a18b44b1cc77a253fbc68de6148e4edd	Become the industry's go-to marketer. Subscribe now!	SUBSCRIBE
23859412891820310	23859412591090310	Subscibe to join 10,000+ readers across India 2023-09-16-d079fff64e545603a1f5fe1b35478540	Subscibe to join 10,000+ readers across India	SUBSCRIBE
23859412907960310	23859412894310310	Subscibe to join 10,000+ readers across India 2023-09

In [4]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model_name="gpt-4o", temperature=0)
generate_query_chain=create_sql_query_chain(llm, db)

query=generate_query_chain.invoke({"question":"which campaign has the most clicks?"})

print(query)


```sql
SELECT `campaigns`.`name`, `campaign_insights`.`clicks`
FROM `adsnerd`.`campaign_insights`
JOIN `adsnerd`.`campaigns` ON `campaign_insights`.`campaign_id` = `campaigns`.`id`
ORDER BY `campaign_insights`.`clicks` DESC
LIMIT 1;
```


In [5]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)
execute_query.invoke(query)


'Error: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near \'```sql\\nSELECT `campaigns`.`name`, `campaign_insights`.`clicks`\\nFROM `adsnerd`.`c\' at line 1")\n[SQL: ```sql\nSELECT `campaigns`.`name`, `campaign_insights`.`clicks`\nFROM `adsnerd`.`campaign_insights`\nJOIN `adsnerd`.`campaigns` ON `campaign_insights`.`campaign_id` = `campaigns`.`id`\nORDER BY `campaign_insights`.`clicks` DESC\nLIMIT 1;\n```]\n(Background on this error at: https://sqlalche.me/e/20/f405)'

In [6]:
chain= generate_query_chain | execute_query
chain.invoke({"question":"Which campaign has the most clicks?"})

'Error: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near \'```sql\\nSELECT \\n    `c`.`name`, \\n    `ci`.`clicks`\\nFROM \\n    `adsnerd`.`campaigns\' at line 1")\n[SQL: ```sql\nSELECT \n    `c`.`name`, \n    `ci`.`clicks`\nFROM \n    `adsnerd`.`campaigns` AS `c`\nJOIN \n    `adsnerd`.`campaign_insights` AS `ci` ON `c`.`id` = `ci`.`campaign_id`\nORDER BY \n    `ci`.`clicks` DESC\nLIMIT 1;\n```]\n(Background on this error at: https://sqlalche.me/e/20/f405)'

In [7]:
chain.get_prompts()[0].pretty_print()

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the S

In [8]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, provide a comprehensive answer following the guidelines below:

You are an expert Meta ads analyst and SQL query specialist. Your task is to interpret user questions about Meta ad performance, analyze SQL query results (including multi-row and multi-column data), and provide comprehensive, data-driven answers.

Follow these steps:
1. Analyze the user's question to understand the core topic and intent.
2. Carefully examine the SQL result, noting the number of rows and columns.
3. If the result is tabular (multiple rows/columns):
   a) Summarize the overall structure of the data (e.g., "The result shows data for 5 ads across 3 metrics").
   b) Identify and highlight key trends or patterns in the data.
   c) Mention the top 3-5 rows or most significant data points, providing context.
   d) Compare and contrast different rows or columns as relevant.
4. Interpret the query results, focusing on key Meta advertising metrics (e.g., CTR, CPC, ROAS, Frequency, Reach).
5. Provide a clear, actionable answer structured as follows:
   a) Summary of findings
   b) Detailed analysis with specific metrics and comparisons
   c) Performance insights and their implications
   d) At least two actionable recommendations based on the data
   e) Suggestions for follow-up analyses or questions
6. Relate your analysis to common Meta advertising objectives (e.g., awareness, consideration, conversion).
7. Consider the impact on different parts of the advertising funnel.

Remember to:
- Use ALL the data available in the query results, not just the top row.
- For tabular data, provide a holistic interpretation that covers the entire dataset.
- Clearly state any assumptions or limitations in your analysis.
- Maintain a professional yet conversational tone.
- Prioritize accuracy, relevance, and actionable insights.
- Use Meta-specific terminology where appropriate (e.g., ad sets, campaigns, placements).
- Consider the broader context of the Meta ads ecosystem (e.g., algorithm learning, audience saturation).
- If the data spans a time period, note any temporal trends or changes.

Question: {question}
SQL Query: {query}
SQL Result: {result}

Answer: """
)

rephrase_answer = answer_prompt | llm | StrOutputParser()

chain = (
    RunnablePassthrough.assign(query=generate_query_chain).assign(
        result=itemgetter("query") | execute_query
     )
     | rephrase_answer
 )

chain.invoke({"question": "Ad with the most clicks?"})

'### Summary of Findings\nThe SQL query intended to identify the ad with the most clicks encountered a syntax error and did not execute successfully. Therefore, we do not have the specific data to determine which ad had the highest number of clicks.\n\n### Detailed Analysis\n1. **SQL Error Explanation**: The error message indicates a syntax issue in the SQL query. Specifically, the error suggests that the backticks and the `sql` keyword used to format the query might be causing the problem. This is not standard SQL syntax and should be removed.\n2. **Corrected SQL Query**: To resolve the syntax error, the query should be simplified to standard SQL format:\n   ```sql\n   SELECT `ad_id`, `clicks`\n   FROM `adsnerd`.`ad_insights`\n   ORDER BY `clicks` DESC\n   LIMIT 1;\n   ```\n\n### Performance Insights and Implications\nSince the query did not execute, we cannot provide specific insights into the ad performance based on clicks. However, identifying the ad with the most clicks is crucial

In [9]:
examples = [
    {
        "input": "What are the top 5 campaigns by ROI in the last 30 days, considering all conversion types?",
        "accountId": "act_624496083171435",
        "query": """
            WITH campaign_performance AS (
                SELECT 
                    c.id,
                    c.name,
                    SUM(ci.spend) AS total_spend,
                    SUM(cc.value) AS total_conversions
                FROM campaigns c
                JOIN campaign_insights ci ON c.id = ci.campaign_id
                LEFT JOIN campaign_conversions cc ON ci.id = cc.campaign_insight_id
                WHERE c.account_id = 'act_624496083171435'
                    AND ci.date_start >= DATE_SUB(CURDATE(), INTERVAL 30 DAY)
                GROUP BY c.id, c.name
            )
            SELECT 
                name AS campaign_name,
                total_spend,
                total_conversions,
                (total_conversions - total_spend) / total_spend * 100 AS roi_percentage
            FROM campaign_performance
            WHERE total_spend > 0
            ORDER BY roi_percentage DESC
            LIMIT 5;
        """,
    },
    {
        "input": "Compare the performance of different ad creative types across all campaigns in terms of CTR, CPC, and conversion rate in the last quarter",
        "accountId": "act_624496083171435",
        "query": """
            WITH ad_performance AS (
                SELECT 
                    ac.call_to_action_type,
                    SUM(ai.impressions) AS total_impressions,
                    SUM(ai.clicks) AS total_clicks,
                    SUM(ai.spend) AS total_spend,
                    SUM(cc.value) AS total_conversions
                FROM ad_creatives ac
                JOIN ads a ON ac.ad_id = a.id
                JOIN ad_insights ai ON a.id = ai.ad_id
                JOIN campaigns c ON a.campaign_id = c.id
                LEFT JOIN campaign_insights ci ON c.id = ci.campaign_id
                LEFT JOIN campaign_conversions cc ON ci.id = cc.campaign_insight_id
                WHERE c.account_id = 'act_624496083171435'
                    AND ai.date_start >= DATE_SUB(CURDATE(), INTERVAL 3 MONTH)
                GROUP BY ac.call_to_action_type
            )
            SELECT 
                call_to_action_type,
                (total_clicks / NULLIF(total_impressions, 0)) * 100 AS ctr,
                total_spend / NULLIF(total_clicks, 0) AS cpc,
                (total_conversions / NULLIF(total_clicks, 0)) * 100 AS conversion_rate
            FROM ad_performance
            ORDER BY conversion_rate DESC;
        """,
    },
    {
        "input": "Analyze the daily budget utilization and performance trends for each campaign objective over the last 2 weeks",
        "accountId": "act_624496083171435",
        "query": """
            WITH daily_performance AS (
                SELECT 
                    c.id AS campaign_id,
                    c.name AS campaign_name,
                    c.objective,
                    c.daily_budget,
                    ci.date_start,
                    SUM(ci.spend) AS daily_spend,
                    SUM(ci.impressions) AS daily_impressions,
                    SUM(ci.clicks) AS daily_clicks,
                    SUM(cc.value) AS daily_conversions
                FROM campaigns c
                JOIN campaign_insights ci ON c.id = ci.campaign_id
                LEFT JOIN campaign_conversions cc ON ci.id = cc.campaign_insight_id
                WHERE c.account_id = 'act_624496083171435'
                    AND ci.date_start >= DATE_SUB(CURDATE(), INTERVAL 2 WEEK)
                GROUP BY c.id, c.name, c.objective, c.daily_budget, ci.date_start
            )
            SELECT 
                campaign_name,
                objective,
                date_start,
                daily_budget,
                daily_spend,
                (daily_spend / daily_budget) * 100 AS budget_utilization_percentage,
                daily_impressions,
                daily_clicks,
                daily_conversions,
                (daily_clicks / NULLIF(daily_impressions, 0)) * 100 AS daily_ctr,
                daily_spend / NULLIF(daily_clicks, 0) AS daily_cpc,
                (daily_conversions / NULLIF(daily_clicks, 0)) * 100 AS daily_conversion_rate
            FROM daily_performance
            ORDER BY campaign_name, date_start;
        """,
    },
    {
        "input": "Identify underperforming ads across all campaigns based on a composite score of CTR, CPC, and conversion rate, compared to campaign averages",
        "accountId": "act_624496083171435",
        "query": """
            WITH ad_metrics AS (
                SELECT 
                    a.id AS ad_id,
                    a.name AS ad_name,
                    a.campaign_id,
                    SUM(ai.impressions) AS impressions,
                    SUM(ai.clicks) AS clicks,
                    SUM(ai.spend) AS spend,
                    SUM(cc.value) AS conversions
                FROM ads a
                JOIN ad_insights ai ON a.id = ai.ad_id
                JOIN campaigns c ON a.campaign_id = c.id
                LEFT JOIN campaign_insights ci ON c.id = ci.campaign_id
                LEFT JOIN campaign_conversions cc ON ci.id = cc.campaign_insight_id
                WHERE c.account_id = 'act_624496083171435'
                GROUP BY a.id, a.name, a.campaign_id
            ),
            campaign_avg_metrics AS (
                SELECT 
                    campaign_id,
                    AVG(clicks / NULLIF(impressions, 0)) AS avg_ctr,
                    AVG(spend / NULLIF(clicks, 0)) AS avg_cpc,
                    AVG(conversions / NULLIF(clicks, 0)) AS avg_cvr
                FROM ad_metrics
                GROUP BY campaign_id
            )
            SELECT 
                am.ad_name,
                am.campaign_id,
                (am.clicks / NULLIF(am.impressions, 0)) AS ctr,
                (am.spend / NULLIF(am.clicks, 0)) AS cpc,
                (am.conversions / NULLIF(am.clicks, 0)) AS cvr,
                cam.avg_ctr,
                cam.avg_cpc,
                cam.avg_cvr,
                (
                    ((am.clicks / NULLIF(am.impressions, 0)) / NULLIF(cam.avg_ctr, 0)) +
                    (cam.avg_cpc / NULLIF((am.spend / NULLIF(am.clicks, 0)), 0)) +
                    ((am.conversions / NULLIF(am.clicks, 0)) / NULLIF(cam.avg_cvr, 0))
                ) / 3 AS performance_score
            FROM ad_metrics am
            JOIN campaign_avg_metrics cam ON am.campaign_id = cam.campaign_id
            WHERE am.impressions > 0
            ORDER BY performance_score ASC
            LIMIT 10;
        """,
    },
]

In [10]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder,FewShotChatMessagePromptTemplate,PromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}\nAccount ID: {accountId}\nSQL Query:"),
        ("ai", "{query}")
    ]
)

few_shot_prompt=FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    input_variables=["input"]
)

print(few_shot_prompt.invoke({"input":"Which ads has the highest CTR?"}))





messages=[HumanMessage(content='What are the top 5 campaigns by ROI in the last 30 days, considering all conversion types?\nAccount ID: act_624496083171435\nSQL Query:'), AIMessage(content="\n            WITH campaign_performance AS (\n                SELECT \n                    c.id,\n                    c.name,\n                    SUM(ci.spend) AS total_spend,\n                    SUM(cc.value) AS total_conversions\n                FROM campaigns c\n                JOIN campaign_insights ci ON c.id = ci.campaign_id\n                LEFT JOIN campaign_conversions cc ON ci.id = cc.campaign_insight_id\n                WHERE c.account_id = 'act_624496083171435'\n                    AND ci.date_start >= DATE_SUB(CURDATE(), INTERVAL 30 DAY)\n                GROUP BY c.id, c.name\n            )\n            SELECT \n                name AS campaign_name,\n                total_spend,\n                total_conversions,\n                (total_conversions - total_spend) / total_spend * 100

In [11]:
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma()
vectorstore.delete_collection()
example_selector = SemanticSimilarityExampleSelector.from_examples(
     examples,
     OpenAIEmbeddings(),
     vectorstore,
     k=2,
     input_keys=["input"],
 )
example_selector.select_examples({"input": "how many ads we have?"})
few_shot_prompt = FewShotChatMessagePromptTemplate(
     example_prompt=example_prompt,
     example_selector=example_selector,
     input_variables=["input","top_k"],
 )
print(few_shot_prompt.format(input="How many ads are there?"))

c:\Sreerag\gen-ai\venv\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


Human: Compare the performance of different ad creative types across all campaigns in terms of CTR, CPC, and conversion rate in the last quarter
Account ID: act_624496083171435
SQL Query:
AI: 
            WITH ad_performance AS (
                SELECT 
                    ac.call_to_action_type,
                    SUM(ai.impressions) AS total_impressions,
                    SUM(ai.clicks) AS total_clicks,
                    SUM(ai.spend) AS total_spend,
                    SUM(cc.value) AS total_conversions
                FROM ad_creatives ac
                JOIN ads a ON ac.ad_id = a.id
                JOIN ad_insights ai ON a.id = ai.ad_id
                JOIN campaigns c ON a.campaign_id = c.id
                LEFT JOIN campaign_insights ci ON c.id = ci.campaign_id
                LEFT JOIN campaign_conversions cc ON ci.id = cc.campaign_insight_id
                WHERE c.account_id = 'act_624496083171435'
                    AND ai.date_start >= DATE_SUB(CURDATE(), INTERVAL 3 M

In [12]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a MySQL expert. Given an input question and an account ID, create a syntactically correct MySQL query to run. Always include the account_id in your WHERE clause for filtering.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions, account IDs, and their corresponding SQL queries."),
        few_shot_prompt,
        ("human", "{input}\nAccount ID: {accountId}"),
    ]
)
generate_query = create_sql_query_chain(llm, db, final_prompt)
chain = (
    RunnablePassthrough.assign(query=generate_query).assign(
        result=itemgetter("query") | execute_query
    )
    | rephrase_answer
)

generated_query = generate_query.invoke(
    {
        "question": "What are the campaigns that we should increase the budget of in last 5 months? by how much? and why do you think we should increase the budget of that particular campaign?",
        "accountId": "act_624496083171435",
    }
)
print("Generated SQL query:")
print(generated_query)
clean_query = generated_query.strip().replace('```sql', '').replace('```', '').strip()

try:
    result = execute_query.invoke(clean_query)
    print("\nQuery result:")
    print(result)

    # Process the result with the answer prompt
    answer = rephrase_answer.invoke(
        {
            "question": "What are the campaigns that we should increase the budget of in last 5 months? by how much? and why do you think we should increase the budget of that particular campaign?",
            "query": clean_query,
            "result": result,
        }
    )
    print("\nInterpreted answer:")
    print(answer)

except Exception as e:
    print(f"\nAn error occurred: {str(e)}")
    import traceback
    traceback.print_exc()

Generated SQL query:
WITH campaign_performance AS (
                SELECT 
                    c.id,
                    c.name,
                    c.daily_budget,
                    SUM(ci.spend) AS total_spend,
                    SUM(ci.impressions) AS total_impressions,
                    SUM(ci.clicks) AS total_clicks,
                    SUM(cc.value) AS total_conversions,
                    (SUM(ci.clicks) / NULLIF(SUM(ci.impressions), 0)) * 100 AS ctr,
                    (SUM(cc.value) / NULLIF(SUM(ci.clicks), 0)) * 100 AS conversion_rate
                FROM campaigns c
                JOIN campaign_insights ci ON c.id = ci.campaign_id
                LEFT JOIN campaign_conversions cc ON ci.id = cc.campaign_insight_id
                WHERE c.account_id = 'act_624496083171435'
                    AND ci.date_start >= DATE_SUB(CURDATE(), INTERVAL 5 MONTH)
                GROUP BY c.id, c.name, c.daily_budget
            )
            SELECT 
                id AS campaign_